In [1]:
!zenml init

Found existing ZenML repository at path 'e:\GitHub\ML_OPS'.
⠋ Initializing ZenML repository at e:\GitHub\ML_OPS.

⠋ Initializing ZenML repository at e:\GitHub\ML_OPS.




In [2]:
import torch
from torch import optim
from pytorch_lightning import LightningModule, Trainer
from torch.utils.data import DataLoader
import torch.nn.functional as F
import mlflow


class ExampleNetwork(LightningModule):
    def __init__(self, model, data_loader, val_data_loader,experimentName):
        super(ExampleNetwork, self).__init__()
        self.mdl: torch.nn.Module = model
        self.data_loader: DataLoader = data_loader
        self.val_data_loader: DataLoader = val_data_loader

        self.lr = 0.001
        self.batch_size = 16

    def forward(self, x: torch.Tensor):
        return self.mdl(x.float())

    def predict(self, x: torch.Tensor):
        with torch.no_grad():
            out = self.forward(x.float())
        return out

    def configure_optimizers(self):
        return optim.Adam(self.parameters(), lr=self.lr)

    def train_dataloader(self):
        return self.data_loader

    def val_dataloader(self):
        return self.val_data_loader

    def training_step(self, batch: torch.Tensor, batch_nb: int):
        x, y = batch
        y = torch.squeeze(y.to(torch.int64))
        loss = F.cross_entropy(self(x.float()), y)
        return loss

    def validation_step(self, batch: torch.Tensor, batch_nb: int):
        x, y = batch
        y = torch.squeeze(y.to(torch.int64))
        loss = F.cross_entropy(self(x.float()), y)
        return loss

    def on_fit_end(self):
        mlflow.end_run()

In [3]:
import torch

from torchsig.models.iq_models.efficientnet.efficientnet import efficientnet_b4


model = torch.load("src/classifiers/best_hae.pt")

classes = ["4ask","8pam","16psk","32qam_cross","2fsk","ofdm-256"]
model_eff = efficientnet_b4(
        pretrained=False,
        num_classes=len(classes)
    )

example_model = ExampleNetwork(model_eff,None,None,None)
example_model = example_model.float().to("cuda")

example_model.load_state_dict(torch.load("src/classifiers/10epochs_classifier.pt"))

<All keys matched successfully>

In [4]:

import torchsig
from torch.utils.data import DataLoader
from torchsig.datasets.modulations import ModulationsDataset
import numpy as np
import torch
import torchsig.transforms as ST
from torchsig.utils.cm_plotter import plot_confusion_matrix
from sklearn.metrics import classification_report
from tqdm import tqdm
import matplotlib
import matplotlib.pyplot as plt

In [ ]:
iq_samples = 1024
samples_per_class = 1000
batch_size = 32
Hae_epochs = 30
Hqa_epochs = 30
classifier_epochs = 10
trainbool = True
eff_net_PATH = f"./src/classifiers/{classifier_epochs}epochs_classifier.pt"
device = "cuda"
layers = 5
input_feature_dim = 2
enc_hidden_sizes = [16, 16, 32, 64, 128, 256]
dec_hidden_sizes = [16, 64, 256, 512, 1024, 2048]
codebook_slots = 64
codeword_dim = 64
codebook_init = "normal"
batch_norm = 1
reset_choice = 1
num_res_blocks = 0
cos_reset = 1
compress = 2
hae_lr = 4e-4
hqa_lr = 4e-4
hae_Cos_coeff = 0.001
hqa_Cos_coeff = 0.7
KL_coeff = 0.1
CL_coeff = 0.005

accuracies = []
num_recons = 1
num_classes = len(classes)
layers = len(model)
data_transform = ST.Compose([
        ST.ComplexTo2D(),
    ])
ds_test = ModulationsDataset(
        classes = classes,
        use_class_idx = True,
        level=0,
        num_iq_samples=iq_samples,
        num_samples=int(len(classes)*samples_per_class)/10,
        include_snr=False,
        transform = data_transform
    )

num_test_examples = len(ds_test)

for j in range(layers):
    for k in range(num_recons): 
        y_raw_preds = np.empty((num_test_examples, num_classes))
        y_preds = np.zeros((num_test_examples,))
        y_true = np.zeros((num_test_examples,))
        hae = model[j]
        hae = hae.float().to(device)
        hae.eval()
        example_model.to(device).eval()
        for i in tqdm(range(0, num_test_examples)):
            # Retrieve data
            idx = i  # Use index if evaluating over full dataset
            
            data, label = ds_test[idx]
            #test_x = hae.reconstruct(data)
            test_x = hae.reconstruct(torch.from_numpy(np.expand_dims(data, 0)).float().to(device))
            #test_x = torch.from_numpy(np.expand_dims(data, 0)).float().to(device)
            # Infer
            #test_x = torch.from_numpy(np.expand_dims(test_x, 0)).float().to(device)
            pred_tmp = example_model.predict(test_x)
            pred_tmp = pred_tmp.cpu().numpy() if torch.cuda.is_available() else pred_tmp
            # Argmax
            y_preds[i] = np.argmax(pred_tmp)
            # Store label
            y_true[i] = label
    
    
        acc = np.sum(np.asarray(y_preds) == np.asarray(y_true)) / len(y_true)
        plot_confusion_matrix(
            y_true,
            y_preds,
            classes=classes,
            normalize=True,
            title="Example Modulations Confusion Matrix\nTotal Accuracy: {:.2f}%".format(
                acc * 100
            ),
            text=False,
            rotate_x_text=60,
            figsize=(10, 10),
        )
        confusionMatrix_save_path = f"./vis/confusion_matrix_layer_{j+1}_recon_{k+1}.png"
        #plt.savefig(confusionMatrix_save_path)
        print(f"Layer {j+1}\nClassification Report: \nAccuracy {acc*100}")
        print(classification_report(y_true, y_preds))
        matplotlib.pyplot.close()
        accuracies.append(acc*100)

In [ ]:
vars(model[3])

In [5]:

from zenml.client import Client

artifact = Client().get_artifact_version('1f20bebb-033a-4f60-9dff-0fda94bb9eb4')
loaded_artifact = artifact.load()



In [7]:
loaded_artifact[3]


HAE(
  (prev_model): HAE(
    (prev_model): HAE(
      (prev_model): HAE(
        (encoder): Encoder(
          (blocks): Sequential(
            (0): Conv1d(2, 8, kernel_size=(3,), stride=(2,), padding=(1,))
            (1): Mish()
            (2): Conv1d(8, 16, kernel_size=(3,), stride=(1,), padding=(1,))
            (3): Mish()
            (4): Conv1d(16, 64, kernel_size=(1,), stride=(1,))
            (5): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          )
        )
        (decoder): Decoder(
          (blocks): Sequential(
            (0): Conv1d(64, 16, kernel_size=(3,), stride=(1,), padding=(1,))
            (1): Mish()
            (2): Upsample()
            (3): Conv1d(16, 8, kernel_size=(3,), stride=(1,), padding=(1,))
            (4): Mish()
            (5): Conv1d(8, 2, kernel_size=(3,), stride=(1,), padding=(1,))
            (6): Tanh()
          )
        )
        (normalize): GlobalNormalization1()
      )
      (encoder): Encode

In [12]:
from torchsummary import summary
summary(loaded_artifact[3].to('cuda'),(2,1024))



----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv1d-1               [-1, 8, 512]              56
              Mish-2               [-1, 8, 512]               0
            Conv1d-3              [-1, 16, 512]             400
              Mish-4              [-1, 16, 512]               0
            Conv1d-5              [-1, 64, 512]           1,088
       BatchNorm1d-6              [-1, 64, 512]             128
           Encoder-7              [-1, 64, 512]               0
GlobalNormalization1-8              [-1, 64, 512]               0
            Conv1d-9               [-1, 8, 256]           1,544
             Mish-10               [-1, 8, 256]               0
           Conv1d-11              [-1, 16, 256]             400
             Mish-12              [-1, 16, 256]               0
           Conv1d-13               [-1, 8, 256]             392
           Conv1d-14              [-1

In [13]:
model = loaded_artifact[3].to('cuda')

model.codebook

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:3                                                                                    │
│                                                                                                  │
│   1 model = loaded_artifact[3].to('cuda')                                                        │
│   2                                                                                              │
│ ❱ 3 model.codebook                                                                               │
│   4                                                                                              │
│                                                                                                  │
│ e:\GitHub\ML_OPS\virtualEnv\lib\site-packages\torch\nn\modules\module.py:1688 in __getattr__     │
│                                                                                                  │
│   1685 │   │   │   modules = self.__dict__['_modules']                                           │
│   1686 │   │   │   if name in modules:                                                           │
│   1687 │   │   │   │   return modules[name]                                                      │
│ ❱ 1688 │   │   raise AttributeError(f"'{type(self).__name__}' object has no attribute '{name}'"  │
│   1689 │                                                                                         │
│   1690 │   def __setattr__(self, name: str, value: Union[Tensor, 'Module']) -> None:             │
│   1691 │   │   def remove_from(*dicts_or_sets):                                                  │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
AttributeError: 'HAE' object has no attribute 'codebook'